<center><h1>2. Vainilla CNNs</h1></center>

In [1]:
# Necessary modules
import pandas as pd
import numpy  as np

from keras import backend as K
from keras.models     import Sequential
from keras.layers     import Dense, Dropout, Activation, Flatten
from keras.layers     import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Using TensorFlow backend.


In [2]:
# Get the data
data = pd.read_csv(filepath_or_buffer="../data/face_image_project/fold_0_data.txt", sep="\t")
data = data.sample(frac=1).reset_index(drop=True)
data.head()

,user_id,original_image,face_id,age,gender,x,y,dx,dy,tilt_ang,fiducial_yaw_angle,fiducial_score
0,37303189@N08,11247266776_aeb2872d39_o.jpg,84,"(0, 2)",m,690,771,949,950,-70,0,86
1,29671106@N00,11192883874_1f4a3c2c2f_o.jpg,198,"(8, 12)",m,240,1619,485,484,180,0,10
2,10406201@N05,9543071846_44fc88d4ac_o.jpg,262,"(38, 43)",f,2416,597,631,631,10,0,126
3,29671106@N00,11193006113_56900fc380_o.jpg,193,"(4, 6)",m,2742,1147,281,281,170,0,20
4,63153065@N07,9427844025_2175926187_o.jpg,230,"(25, 32)",m,1981,1114,484,484,175,-15,19


In [3]:
# Get the data
data1 = pd.read_csv(filepath_or_buffer="../data/face_image_project/fold_1_data.txt", sep="\t")
data1 = data1.sample(frac=1).reset_index(drop=True)
data1.head()

,user_id,original_image,face_id,age,gender,x,y,dx,dy,tilt_ang,fiducial_yaw_angle,fiducial_score
0,10792106@N03,11039274833_219316758d_o.jpg,529,"(15, 20)",m,255,833,261,261,5,-15,62
1,114841417@N06,12068806374_21f6bb040f_o.jpg,481,"(60, 100)",f,997,1177,1046,1046,-90,0,124
2,28754132@N06,9227249084_923a789af0_o.jpg,610,"(8, 12)",m,1392,917,275,275,-85,0,64
3,115153697@N06,12095455223_7bbc11f24f_o.jpg,628,22,f,741,830,357,357,-10,-45,18
4,10897942@N03,10372420736_b5422a0e81_o.jpg,636,"(25, 32)",f,1255,742,1148,1148,0,0,125


In [4]:
# Get the data
data2 = pd.read_csv(filepath_or_buffer="../data/face_image_project/fold_2_data.txt", sep="\t")
data2 = data2.sample(frac=1).reset_index(drop=True)
data2.head()
data2.shape

(3894, 12)

In [5]:
# Get the data
data3 = pd.read_csv(filepath_or_buffer="../data/face_image_project/fold_3_data.txt", sep="\t")
data3 = data3.sample(frac=1).reset_index(drop=True)
data3.head()
data3.shape

(3446, 12)

In [6]:
# Get the data
data4 = pd.read_csv(filepath_or_buffer="../data/face_image_project/fold_4_data.txt", sep="\t")
data4 = data4.sample(frac=1).reset_index(drop=True)
data4.head()
data4.shape

(3816, 12)

In [7]:
data = pd.concat([data,data1,data2,data3,data4], ignore_index=True)
data.shape
data.head()
#data.shape

,user_id,original_image,face_id,age,gender,x,y,dx,dy,tilt_ang,fiducial_yaw_angle,fiducial_score
0,37303189@N08,11247266776_aeb2872d39_o.jpg,84,"(0, 2)",m,690,771,949,950,-70,0,86
1,29671106@N00,11192883874_1f4a3c2c2f_o.jpg,198,"(8, 12)",m,240,1619,485,484,180,0,10
2,10406201@N05,9543071846_44fc88d4ac_o.jpg,262,"(38, 43)",f,2416,597,631,631,10,0,126
3,29671106@N00,11193006113_56900fc380_o.jpg,193,"(4, 6)",m,2742,1147,281,281,170,0,20
4,63153065@N07,9427844025_2175926187_o.jpg,230,"(25, 32)",m,1981,1114,484,484,175,-15,19


In [8]:
data = data[["user_id","original_image","face_id","gender"]]
data.head()

,user_id,original_image,face_id,gender
0,37303189@N08,11247266776_aeb2872d39_o.jpg,84,m
1,29671106@N00,11192883874_1f4a3c2c2f_o.jpg,198,m
2,10406201@N05,9543071846_44fc88d4ac_o.jpg,262,f
3,29671106@N00,11193006113_56900fc380_o.jpg,193,m
4,63153065@N07,9427844025_2175926187_o.jpg,230,m


In [9]:
tem = data[data.gender != "u"]
tem.gender.value_counts()
#
#print data["gender"].value_counts()
#data.head()


f    9372
m    8120
Name: gender, dtype: int64

In [10]:
data = tem
data = data.dropna()
data.gender.isnull().sum()

0

In [11]:
data.loc[data.gender == "f", "gender"] = 0
data.loc[data.gender == "m", "gender"] = 1
data.head()

/usr/local/lib/python2.7/dist-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,user_id,original_image,face_id,gender
0,37303189@N08,11247266776_aeb2872d39_o.jpg,84,1
1,29671106@N00,11192883874_1f4a3c2c2f_o.jpg,198,1
2,10406201@N05,9543071846_44fc88d4ac_o.jpg,262,0
3,29671106@N00,11193006113_56900fc380_o.jpg,193,1
4,63153065@N07,9427844025_2175926187_o.jpg,230,1


In [12]:
path_template = "../data/face_image_project/aligned/%s/landmark_aligned_face.%d.%s"
data["file_path"] = data[["user_id","face_id","original_image"]].apply(lambda x:  
                                                                   path_template % (x[0],x[1],x[2]),
                                                                   axis=1)
data.head()

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,user_id,original_image,face_id,gender,file_path
0,37303189@N08,11247266776_aeb2872d39_o.jpg,84,1,../data/face_image_project/aligned/37303189@N0...
1,29671106@N00,11192883874_1f4a3c2c2f_o.jpg,198,1,../data/face_image_project/aligned/29671106@N0...
2,10406201@N05,9543071846_44fc88d4ac_o.jpg,262,0,../data/face_image_project/aligned/10406201@N0...
3,29671106@N00,11193006113_56900fc380_o.jpg,193,1,../data/face_image_project/aligned/29671106@N0...
4,63153065@N07,9427844025_2175926187_o.jpg,230,1,../data/face_image_project/aligned/63153065@N0...


In [13]:
data = data[["file_path","gender"]]
data.head()

,file_path,gender
0,../data/face_image_project/aligned/37303189@N0...,1
1,../data/face_image_project/aligned/29671106@N0...,1
2,../data/face_image_project/aligned/10406201@N0...,0
3,../data/face_image_project/aligned/29671106@N0...,1
4,../data/face_image_project/aligned/63153065@N0...,1


In [14]:
data["gender"].value_counts()

0    9372
1    8120
Name: gender, dtype: int64

Occurences of each gender seem pretty balanced. Looks good to me

In [15]:
Y      = data["gender"]
X_path = data["file_path"]

In [16]:
X_path[0]

'../data/face_image_project/aligned/37303189@N08/landmark_aligned_face.84.11247266776_aeb2872d39_o.jpg'

In [17]:
tempo = X_path[1:3]
tempo.apply(lambda x: img_to_array( load_img(x) ) ) #DOES IT NEED A RESHAPE

1    [[[63.0, 34.0, 26.0], [66.0, 37.0, 29.0], [68....
2    [[[20.0, 17.0, 24.0], [20.0, 17.0, 24.0], [20....
Name: file_path, dtype: object

In [18]:
import scipy.misc

i_width  = 227
i_height = 227

#scipy.misc.imresize(original_image, (i_height, i_width))


def train_generator(data):
    while True:
        start, end = 0, 32
        while end < len(data):
            data = data.sample(frac=1).reset_index(drop=True)
            sample  = data[start:end]

            X = pd.DataFrame(sample["file_path"].apply(lambda x:  img_to_array( scipy.misc.imresize(load_img(x), (i_height, i_width) ) ) ))
            
            X = X["file_path"].apply(lambda x: x.reshape((1,)+ x.shape))
            X = np.vstack(X)
            #print X[0].shape
            #print X.shape
            #
            Y = sample["gender"].as_matrix()
            #print Y.as_matrix()
            #print X.shape
            #print X
            yield (X, Y)
            start += 32
            end += 32

In [19]:
gen = train_generator(data)
a = gen.next()

In [20]:
# Building a vainilla CNN
vainilla_cnn = Sequential()
vainilla_cnn.add(Convolution2D(32, 3, 3, input_shape=(227,227,3)))
vainilla_cnn.add(Activation('relu'))
vainilla_cnn.add(MaxPooling2D(pool_size=(2, 2)))
vainilla_cnn.add(Convolution2D(64, 3, 3))
vainilla_cnn.add(Activation('relu'))
vainilla_cnn.add(Convolution2D(64, 3, 3))
vainilla_cnn.add(Activation('relu'))
vainilla_cnn.add(Dropout(0.3))
vainilla_cnn.add(MaxPooling2D(pool_size=(2, 2)))
vainilla_cnn.add(Convolution2D(64, 3, 3))
vainilla_cnn.add(Activation('relu'))
vainilla_cnn.add(MaxPooling2D(pool_size=(2, 2)))
vainilla_cnn.add(Dropout(0.5))
vainilla_cnn.add(Flatten())
vainilla_cnn.add(Dense(output_dim=200, input_dim=500))
vainilla_cnn.add(BatchNormalization())
vainilla_cnn.add(Activation("relu"))
vainilla_cnn.add(Dense(output_dim=100, input_dim=200))
vainilla_cnn.add(BatchNormalization())
vainilla_cnn.add(Activation("relu"))
vainilla_cnn.add(Dropout(0.5))
vainilla_cnn.add(Dense(output_dim=1))
vainilla_cnn.add(Activation("sigmoid"))

In [21]:
# Compile
vainilla_cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [22]:
# Train
vainilla_cnn.fit_generator(train_generator(data), samples_per_epoch=64, nb_epoch=2069)

Epoch 1/2069
64/64 [==============================] - 6s - loss: 0.7963 - acc: 0.5000     
Epoch 2/2069
64/64 [==============================] - 1s - loss: 0.7999 - acc: 0.6094     
Epoch 3/2069
64/64 [==============================] - 1s - loss: 0.8040 - acc: 0.6094     
Epoch 4/2069
64/64 [==============================] - 1s - loss: 0.6581 - acc: 0.5938     
Epoch 5/2069
64/64 [==============================] - 1s - loss: 0.5822 - acc: 0.6406     
Epoch 6/2069
64/64 [==============================] - 1s - loss: 0.7330 - acc: 0.6562     

In [23]:
!nvidia-smi

Sun Mar 12 00:35:07 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 367.57                 Driver Version: 367.57                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GRID K520           On   | 0000:00:03.0     Off |                  N/A |
| N/A   64C    P0    79W / 125W |   3801MiB /  4036MiB |    100%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [24]:
vainilla_cnn.layers[0].name

'convolution2d_1'

In [25]:
# util function to convert a tensor into a valid image
def deprocess_image(x):
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255

    if x.shape[2] != 3:
        x = x.transpose((1, 2, 0))
    
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [26]:
vainilla_cnn.layers[0].input_shape

(None, 227, 227, 3)

In [27]:
layer_dict = dict([(layer.name, layer) for layer in vainilla_cnn.layers])
layer_dict

{'activation_1': <keras.layers.core.Activation at 0x7feb48a3ffd0>,
 'activation_2': <keras.layers.core.Activation at 0x7feb489831d0>,
 'activation_3': <keras.layers.core.Activation at 0x7feb489bf110>,
 'activation_4': <keras.layers.core.Activation at 0x7feb488d2e90>,
 'activation_5': <keras.layers.core.Activation at 0x7feb4881ff90>,
 'activation_6': <keras.layers.core.Activation at 0x7feb4870cfd0>,
 'activation_7': <keras.layers.core.Activation at 0x7feb48606f50>,
 'batchnormalization_1': <keras.layers.normalization.BatchNormalization at 0x7feb48870c90>,
 'batchnormalization_2': <keras.layers.normalization.BatchNormalization at 0x7feb487be890>,
 'convolution2d_1': <keras.layers.convolutional.Convolution2D at 0x7feb48a3ff50>,
 'convolution2d_2': <keras.layers.convolutional.Convolution2D at 0x7feb489c7f50>,
 'convolution2d_3': <keras.layers.convolutional.Convolution2D at 0x7feb489b0f90>,
 'convolution2d_4': <keras.layers.convolutional.Convolution2D at 0x7feb489a2b90>,
 'dense_1': <keras.

In [28]:
def iterateForFilter(layer_dict, layer_name, filter_idx):
    first_layer = vainilla_cnn.layers[0]
    # this is a placeholder tensor that will contain our generated images
    input_img = first_layer.input

    loss = K.mean(layer_dict[layer_name].output[:,:,:, filter_idx])

    # compute the gradient of the input picture wrt this loss
    grads = K.gradients(loss, input_img)[0]

    # normalization trick: we normalize the gradient
    grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5)

    # this function returns the loss and grads given the input picture
    iterate = K.function([input_img], [loss, grads])
    
    return iterate

In [29]:
layer_dict['convolution2d_1'].output_shape[3]

32

In [35]:
img_width = 227
img_height = 227

input_img_data = np.random.random((1, img_width, img_height, 3)) * 20 + 128.

#layers_to_visualize = ['convolution2d_1', 'convolution2d_2', 'convolution2d_3']
layers_to_visualize = ['convolution2d_4']

for layer_name in layers_to_visualize:
    
    num_filters = layer_dict[layer_name].output_shape[3]
    print "getting outputs for layer %s with %d filters" % (layer_name, num_filters)
    
    for i in range(num_filters):
        iterate = iterateForFilter(layer_dict, layer_name, i)       
        # we start from a gray image with some noise
        
        # run gradient ascent for 100 steps
        step = 1
        for _ in range(200):
            loss_value, grads_value = iterate([input_img_data])
            input_img_data += grads_value * step
            from scipy.misc import imsave

        img = input_img_data[0]
        img = deprocess_image(img)
        imsave('%s_filter_%d.png' % (layer_name, i), img)

getting outputs for layer convolution2d_4 with 64 filters


InvalidArgumentError: You must feed a value for placeholder tensor 'keras_learning_phase' with dtype bool
	 [[Node: keras_learning_phase = Placeholder[dtype=DT_BOOL, shape=[], _device="/job:localhost/replica:0/task:0/gpu:0"]()]]
	 [[Node: div_188/_2747 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_49_div_188", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'keras_learning_phase', defined at:
  File "/usr/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_i

In [31]:
batch1 = train_generator().next()


TypeError: train_generator() takes exactly 1 argument (0 given)